In [ ]:
%matplotlib inline

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nest
import numpy as np

In [ ]:
# simulation parameters
T = 1000.                  # simulation time (ms)
dt = 0.1                   # simulation resolution (ms)

# network parameters
N_L5E = 4850               # number of neurons in L5E
N_L5I = 1065               # number of neurons in L5I
    
Nsyn_L5E_L5E = 2038173     # number of synapses with presynaptic neuron (pre) in L5E and postsynaptic neuron (post) in L5E 
Nsyn_L5E_L5I = 319602      # number of synapses with pre in L5E and post in L5I
Nsyn_L5I_L5I = 430775      # number of synapses with pre in L5I and post in L5I
Nsyn_L5I_L5E = 2411184     # number of synapses with pre in L5I and post in L5E

K_L5E_ext = 2000           # indegree of excitatory neurons to external poisson drive
K_L5I_ext = 1900           # indegree of inhibitory neurons to external poisson drive

# neuron parameters
neuron_params = {
    'C_m'       : 250.,    # pF
    'I_e'       : 0.0,     # nA
    'tau_m'     : 10.0,    # ms
    't_ref'     : 2.0,     # ms
    'tau_syn_ex': 0.5,     # ms
    'tau_syn_in': 0.5,     # ms
    'V_reset'   : -65.0,   # mV
    'E_L'       : -65.0,   # mV
    'V_th'      : -50.0    # mV
}

# synapse parameters
w = 87.8                   # mean excitatory weight (pA)
sigma_w = 8.8              # standard deviation of excitatory weight (pA)
g = -4.                    # relative inhibitory weight 

de = 1.5                   # mean spike transmission delay for excitatory presynaptic neurons (ms)
sigma_de = 0.75            # standard deviation 
di = 0.8                   # mean spike transmission delay for inhibitory presynaptic neurons (ms)
sigma_di = 0.4             # standard deviation
 
# input parameters
bg_rate = 8.               # external poisson rate
perturbation = False

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'resolution': dt})      # set simulation resolution

In [ ]:
# create excitatory and inhibitory neuron populations
pop_L5E = nest.Create('iaf_psc_exp', N_L5E)
nest.SetStatus(pop_L5E, params=neuron_params)

pop_L5I = nest.Create('iaf_psc_exp', N_L5I, params=neuron_params)

In [ ]:
# excitatory connections (connections from L5E)

# specifying the synapse parameters
syn_dict = {
    'synapse_model': 'static_synapse',
    'weight': nest.math.redraw(
        nest.random.normal(
            mean=w,
            std=sigma_w),
        min=0,
        max=np.Inf),
    'delay': nest.math.redraw(
                nest.random.normal(
            mean=de,
            std=sigma_de),
        min=nest.resolution, # TODO
        max=np.Inf)
}

# connections to L5E
# specifying the connection parameters
conn_dict = {'rule': 'fixed_total_number', 'N': Nsyn_L5E_L5E}
nest.Connect(pop_L5E, pop_L5E, conn_dict, syn_dict)

# connections to L5I
conn_dict = {'rule': 'fixed_total_number', 'N': Nsyn_L5E_L5I}
nest.Connect(pop_L5E, pop_L5I, conn_dict, syn_dict)

In [ ]:

# inhibitory connections (connections from L5I)

# specifying the synapse parameters
syn_dict = {
    'synapse_model': 'static_synapse',
    'weight': nest.math.redraw(
        nest.random.normal(
            mean=g*w,
            std=np.abs(g*sigma_w)),
        min=-np.Inf,
        max=0),
    'delay': nest.math.redraw(
                nest.random.normal(
            mean=di,
            std=sigma_di),
        min=nest.resolution, # TODO
        max=np.Inf)
}

# connections to L5E
# specifying the connection parameters
conn_dict = {'rule': 'fixed_total_number', 'N': Nsyn_L5I_L5E}
nest.Connect(pop_L5I, pop_L5E, conn_dict, syn_dict)

# connections to L5I
conn_dict = {'rule': 'fixed_total_number', 'N': Nsyn_L5I_L5I}
nest.Connect(pop_L5I, pop_L5I, conn_dict, syn_dict)
     

In [ ]:
# connections with external poisson generator

# connections to L5E
poisson_generator_L5E = nest.Create('poisson_generator',
                                    params={'rate': bg_rate*K_L5E_ext})
nest.Connect(poisson_generator_L5E, pop_L5E, 'all_to_all', {'weight': w, 'delay': de})

# connections to L5I
poisson_generator_L5I = nest.Create('poisson_generator',
                                    params={'rate': bg_rate*K_L5I_ext})
nest.Connect(poisson_generator_L5I, pop_L5I, 'all_to_all', {'weight': w, 'delay': de})

In [ ]:
# set up and connect spike detector
sd_L5E = nest.Create('spike_recorder')
nest.Connect(pop_L5E, sd_L5E, 'all_to_all')

sd_L5I = nest.Create('spike_recorder')
nest.Connect(pop_L5I, sd_L5I, 'all_to_all')

In [ ]:
# run simulation
nest.Simulate(T)    

In [ ]:
# read out recorded spikes
spike_senders_L5E = nest.GetStatus(sd_L5E)[0]['events']['senders']
spike_times_L5E = nest.GetStatus(sd_L5E)[0]['events']['times']
spike_senders_L5I = nest.GetStatus(sd_L5I)[0]['events']['senders']
spike_times_L5I = nest.GetStatus(sd_L5I)[0]['events']['times']

spike_senders = np.hstack((spike_senders_L5E, spike_senders_L5I))
spike_times = np.hstack((spike_times_L5E, spike_times_L5I))

In [ ]:
# plotting
plt.plot(spike_times, spike_senders, 'k.', markersize=1)
plt.xlim(0, T)
plt.ylim(0, N_L5E+N_L5I)
plt.xlabel('time (ms)')
plt.ylabel('neuron id')